In [1]:
import numpy as np
import tensorflow as tf

In [2]:
tf.__version__

'2.15.0'

# First, let's forge 3 regression problems (tasks). The first regression problem has 3 outputs; second has 2 outputs; third has 5 outputs. All tasks' samples are represented as 20D vector.

In [3]:
T = 3 # number of tasks
O = [2, 3, 5] # number of outputs for every task
N = 100 # number of training samples
D = 20 # dimension of feature vector -- assumed to be the same for all tasks

In [4]:
X = [np.random.randn(N,D) for _ in range(T)]
Y = [x.dot(np.random.randn(D,o)) for x,o in zip(X,O)]

In [5]:
X

[array([[-0.76369853,  2.34956924, -0.53781565, ...,  0.50579801,
         -1.83551257, -0.39640998],
        [ 1.70006712,  1.1859365 , -0.15087725, ...,  0.38154842,
          0.33506636,  0.83509247],
        [ 0.25860875, -0.62302737, -0.20790201, ...,  0.37986871,
          0.08938003,  0.01176639],
        ...,
        [-0.44822243, -0.17323018, -0.21118097, ..., -0.68547707,
         -1.19061395, -0.42368123],
        [ 0.4100562 ,  0.9908569 , -1.88253018, ...,  0.69330531,
         -1.29459484, -0.88616652],
        [-0.08308636,  0.18076164, -0.48637216, ...,  0.24676009,
          0.16486028,  0.73959962]]),
 array([[ 0.38233972, -0.90231136,  0.76839419, ..., -0.77923241,
         -1.03860043,  0.39787046],
        [ 0.19478399,  0.09430632, -0.43658221, ...,  0.38072316,
          0.01730283, -0.13041265],
        [-0.31869368, -0.71844752, -1.24316025, ..., -0.11705214,
          0.15303035, -0.38070588],
        ...,
        [ 0.45953529, -0.33544402,  0.10028427, ...,  

In [6]:
X[1].shape

(100, 20)

In [7]:
Y

[array([[ 6.77230873e+00, -2.48862088e+00],
        [ 5.45056314e+00, -9.91140145e-02],
        [ 3.65174497e+00,  5.40773545e-01],
        [-5.57666394e+00,  1.28871854e+00],
        [-3.18418168e+00, -3.44711087e+00],
        [ 4.30715552e+00, -3.05074887e+00],
        [ 5.21182488e-01, -1.80546994e-01],
        [ 6.35512226e+00,  1.63336174e+00],
        [-8.48595386e+00,  3.20927168e+00],
        [-1.56911228e+00, -6.19717590e+00],
        [-4.03347297e+00,  3.33296365e+00],
        [-1.26018775e+00, -7.63369599e-01],
        [ 5.87767313e+00, -3.41351566e-02],
        [ 1.15927720e+00, -2.00680017e+00],
        [-5.01904598e+00,  1.83983761e+00],
        [ 4.55576234e+00,  1.81756299e+00],
        [-3.10726060e+00, -3.05446442e-01],
        [-2.12629151e-01, -3.75154907e+00],
        [ 4.17432333e+00, -1.71364100e+00],
        [ 2.87064659e+00,  3.17469548e+00],
        [-4.03320080e+00,  2.71421284e+00],
        [ 2.93163001e+00, -3.70452943e+00],
        [ 4.90553622e+00, -5.880

In [8]:
Y[1].shape

(100, 3)

# Second, let's train a regression model for each task independently. Regression model is realised by a 3-layer neuron network, and the hidden layer has 10 neurons

In [9]:
sess=tf.compat.v1.InteractiveSession()

In [10]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

Instructions for updating:
non-resource variables are not supported in the long term


In [11]:
H = 10
W_input_to_hidden = [tf.Variable(tf.random.truncated_normal(shape=[D, H])) for _ in range(T)]
b_input_to_hidden = [tf.Variable(tf.zeros(shape=[H])) for _ in range(T)]
W_hidden_to_output = [tf.Variable(tf.random.truncated_normal(shape=[H, o])) for o in O]
b_hidden_to_output = [tf.Variable(tf.zeros(shape=[o])) for o in O]

In [12]:
X_placeholder = [tf.compat.v1.placeholder(tf.float32, shape=(None, D)) for _ in range(T)]
Y_placeholder = [tf.compat.v1.placeholder(tf.float32, shape=[None, o]) for o in O]

In [13]:
Y_hat = [tf.nn.xw_plus_b(tf.nn.sigmoid(tf.nn.xw_plus_b(x,w0,b0)),w1,b1)
         for x,w0,b0,w1,b1 in zip(X_placeholder, W_input_to_hidden, b_input_to_hidden, W_hidden_to_output, b_hidden_to_output)]

In [14]:
MSE = [tf.reduce_mean(tf.squared_difference(y,y_hat)) for y,y_hat in zip(Y_placeholder,Y_hat)]

In [15]:
loss = tf.reduce_mean(MSE)

In [16]:
opt = tf.train.AdamOptimizer(learning_rate=0.01)

In [17]:
train = opt.minimize(loss)

In [18]:
sess.run(tf.global_variables_initializer())

In [19]:
feed_dict = dict(list(zip(X_placeholder,X))+list(zip(Y_placeholder,Y)))

In [20]:
for _ in range(1000):
    train.run(feed_dict=feed_dict)
    if _ % 100 == 0:
        print(loss.eval(feed_dict=feed_dict))

17.526104
4.7661324
1.5166807
0.7461162
0.43377605
0.29200947
0.21086477
0.16285688
0.1299688
0.10644266


In [21]:
# We can resue the parameters learned above for initialisation, though it is optional
W_init = np.stack(sess.run(W_input_to_hidden))

In [22]:
# We put the task-axis in the last position
W_init = np.transpose(W_init, axes=[1,2,0])

In [23]:
sess.close()

In [24]:
tf.reset_default_graph()

In [25]:
sess = tf.InteractiveSession()

In [26]:
import numpy as np
import tensorflow as tf
from tensorflow.python.framework import function
from tensorflow.python.framework import dtypes


@function.Defun(dtypes.float32, dtypes.float32)
def nuclear_norm_grad(x, dy):
    _, U, V = tf.linalg.svd(x, full_matrices=False, compute_uv=True)
    grad = tf.matmul(U, tf.transpose(V))
    return dy * grad


@function.Defun(dtypes.float32, grad_func=nuclear_norm_grad)
def nuclear_norm(x):
    sigma = tf.linalg.svd(x, full_matrices=False, compute_uv=False)
    norm = tf.reduce_sum(sigma)
    return norm


def TensorUnfold(A, k):
    tmp_arr = np.arange(A.get_shape().ndims)
    A = tf.transpose(A, [tmp_arr[k]] + np.delete(tmp_arr, k).tolist())
    shapeA = A.get_shape().as_list()
    A = tf.reshape(A, [shapeA[0], np.prod(shapeA[1:])])
    return A


def TensorTraceNorm(X, method='Tucker'):
    shapeX = X.get_shape().as_list()
    dimX = len(shapeX)
    if method == 'Tucker':
        re = [nuclear_norm(i) for i in [TensorUnfold(X, j) for j in range(dimX)]]
    elif method == 'TT':
        re = [nuclear_norm(i) for i in
              [tf.reshape(X, [np.prod(shapeX[:j]), np.prod(shapeX[j:])]) for j in range(1, dimX)]]
    elif method == 'LAF':
        re = [nuclear_norm(TensorUnfold(X, -1))]
    return tf.stack(re)

In [27]:
# Here we use "TensorTraceNorm" to get the trace norm of shareable layer's parameter (tensor)
W_input_to_hidden = tf.Variable(tf.random.truncated_normal(shape=[D, H, T]))
Trace_norm_input_to_hidden = TensorTraceNorm(W_input_to_hidden, 'LAF') # Three methods: 'LAF', 'Tucker', and 'TT'
W_input_to_hidden = [W_input_to_hidden[:,:,i] for i in range(T)]

In [28]:
# Nothing changes for bias terms or unshared parameters
b_input_to_hidden = [tf.Variable(tf.zeros(shape=[H])) for _ in range(T)]
W_hidden_to_output = [tf.Variable(tf.random.truncated_normal(shape=[H, o])) for o in O]
b_hidden_to_output = [tf.Variable(tf.zeros(shape=[o])) for o in O]

In [29]:
# Build the network as usual
X_placeholder = [tf.compat.v1.placeholder(tf.float32, shape=[None, D]) for _ in range(T)]
Y_placeholder = [tf.compat.v1.placeholder(tf.float32, shape=[None, o]) for o in O]

Y_hat = [tf.compat.v1.nn.xw_plus_b(tf.nn.sigmoid(tf.compat.v1.nn.xw_plus_b(x,w0,b0)),w1,b1)
         for x,w0,b0,w1,b1 in zip(X_placeholder, W_input_to_hidden, b_input_to_hidden, W_hidden_to_output, b_hidden_to_output)]

MSE = [tf.reduce_mean(tf.math.squared_difference(y,y_hat)) for y,y_hat in zip(Y_placeholder,Y_hat)]

In [30]:
# Here we add trace norm as part of losses
loss = tf.reduce_mean(MSE) + 0.001 * tf.reduce_sum(Trace_norm_input_to_hidden)

In [31]:
opt = tf.compat.v1.train.AdamOptimizer(learning_rate=0.01)

train = opt.minimize(loss)

In [32]:
# Train the model as usual
init_op = tf.compat.v1.global_variables_initializer() # Workaround: https://github.com/tensorflow/tensorflow/issues/6804
sess.run(init_op)

In [33]:
feed_dict = dict(list(zip(X_placeholder,X))+list(zip(Y_placeholder,Y)))

In [34]:
for _ in range(1000):
    train.run(feed_dict=feed_dict)
    if _ % 100 == 0:
        print(loss.eval(feed_dict=feed_dict))

19.251493
4.670026
1.6094794
0.8626469
0.55489045
0.39467344
0.30067885
0.23889652
0.19778669
0.16817752
